https://www.kaggle.com/krishanudb/pytorch-tutorial-for-beginners  

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # The convolutional layers are defined. Note that these are all functions but are defined like attributes
        # input channels = 1, output channels = 6, window size = 5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # input channels = 6, output channels = 16, window size = 5
        self.conv2 = nn.Conv2d(6, 16, 5) 
        # The fully connected layers are defined
        # input nodes = 16 * 5 * 5 (since each of the 16 channels are of the size 5 * 5), output = 120
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # input = 120 output = 84
        self.fc2 = nn.Linear(120, 84)
        # last layer: input = 84, output = 10
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # The first convolutional layer with all the max pooling and relu layers: there is a 2X2 max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # The second conv layer: 2 X 2 max pooling
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # The reshape (flattening) layer which changes the shape of the tensor from that of a multichannel 2D tensor to a 1D tensor
        x = x.view(-1, self.num_flat_features(x))
        # Fully connected layer with activation
        x = F.relu(self.fc1(x))
        # Fully connected layer with activation
        x = F.relu(self.fc2(x))
        # Output layer
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # The shape of the tensor without the first dimension which defines the number of samples
        size = x.size()[1:]
        # the shapes are are multiplied together to give the number of features
        num_features = 1
        for s in size: num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
params = list(net.parameters())
print(len(params))
for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [9]:
# Now, we define a simple random tensor and compute the output
inp = torch.randn(1, 1, 32, 32)
out = net(inp)
print(out)

tensor([[ 0.1544,  0.0610,  0.1013, -0.0285, -0.0681,  0.0851, -0.1465,  0.0749,
          0.0215, -0.1236]], grad_fn=<AddmmBackward>)


In [10]:
# We zero the gradient buffers of all the parameters
net.zero_grad()

In [11]:
# The backprops are initialized with random gradients
out.backward(torch.randn(1, 10))

In [12]:
# Now we define the loss function.
out = net(inp)
target = torch.randn(10)
target = target.view(1, -1)
criteria = nn.MSELoss()
loss = criteria(out, target)
print(loss)

tensor(0.6018, grad_fn=<MseLossBackward>)


In [13]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [14]:
net.zero_grad()
print("Conv 1 bias before backward()")
print(net.conv1.bias.grad)
loss.backward()
print("Conv 2 bias after backward()")
print(net.conv1.bias.grad)

Conv 1 bias before backward()
tensor([0., 0., 0., 0., 0., 0.])
Conv 2 bias after backward()
tensor([ 0.0128,  0.0142,  0.0032,  0.0220, -0.0196,  0.0058])


In [15]:
learning_rate = 0.001
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [16]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr = 0.01)
optimizer.zero_grad()
out = net(inp)
loss = criteria(out, target)
loss.backward()
optimizer.step()